In [ ]:
import csv
import gzip
import os
filepaths = {}

with open('/40TB_3/InovaGenomes/IlluminaList') as gvcfFileList:
    gvcfListReader = csv.reader(gvcfFileList, delimiter = '\t')
    for row in gvcfListReader:
        genome_id = row[0][9:26]
        filename = genome_id + ".norm.sorted.rcf.gz"
        filepath = os.path.join("/40TB_3/InovaGenomes/" , "RCF/NORM/20150920/", filename)
        filepaths[genome_id] = filepath

In [ ]:
#Takes in two strings, one from the format column and one from the column that 
#specifies the data. Example inputs: 'GT:GQX:DP:DPF' as format_string, and '0/0:21:8:0'
#as data_string. Splits these strings and returns a string that represents the genotype
#enclosed in brackets. 
def get_genotype(format_string, data_string):
    format_list = format_string.split(":")
    data_list = data_string.split(":")
    info = {} #a dict of format subtype string --> data subtype string
    for f, d in zip(format_list, data_list):
        info[f] = d
    return '[' + info['GT'] + ']'

In [ ]:
for genome_id, filepath in filepaths.iteritems(): 
    line_count = {} # dictionary of 'filter' --> count, 
    #or if the filter is 'PASS' then 'PASS[GT]' --> count
    with gzip.open(filepath) as gvcfFile: 
        gvcf_reader = csv.reader(gvcfFile, delimiter = '\t')
        col_labels = ['CHROM', 'START', 'END', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample']
        indexes = range(0, len(col_labels))
        col_dict= {}
        for index, label in zip(indexes, col_labels):
            col_dict[label] = index
        for j, row in enumerate(gvcf_reader):
            if(j > 109): #the first 109 lines are lines we don't need
                filterT = row[col_dict['FILTER']]
                key = filterT
                if(filterT == 'PASS'):
                    formatD = row[col_dict['FORMAT']]
                    data = row[col_dict['Sample']]
                    key += get_genotype(formatD, data)
        
                if line_count.has_key(key):
                    line_count[key] = line_count.get(key) + 1
                else:
                    line_count[key] = 1
            if(j >= 209): break #goes through the first 100 lines of table 
        #print('ID:' + genome_id, line_count)
    
    output_file_name = os.path.join('/40TB_3/gestalt/private/IndividualGenomeAssembly/', genome_id, 'analyses/FileCategoryCount.rcf')
    #print(output_file_name)
    with open(output_file_name, 'wb') as output_file:
        output_writer = csv.writer(output_file, delimiter = '\t')
        output_writer.writerow(['filepath of original rcf file:',filepath]) 
        for key in line_count:
            output_writer.writerow([key, line_count.get(key)])